In [1]:
with open('Input15') as f:
    data = f.read()

In [2]:
import regex as re

In [3]:
from tqdm import tqdm

In [4]:
data = data.split('\n')
data.remove('')

In [5]:
class Sensor:
    
    def __init__(self, coord: tuple):
        
        self.xy_s = ( int(coord[0]), int(coord[1]) )
        self.xy_b = ( int(coord[2]), int(coord[3]) )

In [6]:
def list_sensors(data: list) -> list:
    
    m = re.compile(r"Sensor at x=([-]?\d+), y=([-]?\d+): closest beacon is at x=([-]?\d+), y=([-]?\d+)")
    sensors = [Sensor(m.match(l).groups()) for l in data]
    
    return sensors

In [7]:
def manhattan_distance(pt1 : tuple, pt2: tuple) -> int:
    
    distance = abs(pt2[0]-pt1[0]) + abs(pt2[1]-pt1[1])
    
    return distance

In [8]:
def no_beacon(y: int, data: list)-> int:
    
    sensors = list_sensors(data)
    
    no_beacon = 0
        
    min_x = min(s.xy_s[0] - manhattan_distance(s.xy_s , s.xy_b) for s in sensors)
    max_x = max(s.xy_s[0] + manhattan_distance(s.xy_s , s.xy_b) for s in sensors)
    
    for x in range(min_x, max_x):
        for s in sensors:
            if manhattan_distance((x,y), s.xy_s) <= manhattan_distance(s.xy_s, s.xy_b):                
                if x != s.xy_b[0] or y != s.xy_b[1]:
                    no_beacon += 1
                    break
                    
    return no_beacon

In [9]:
result = no_beacon(2000000, data)

In [10]:
result # résultat partie 1 : 4'883'971

4883971

In [11]:
def intersection_droites(d1: tuple, d2: tuple) -> tuple:
    
    a1 = d1[0]
    b1 = d1[1]
    a2 = d2[0]
    b2 = d2[1]
    
    if a1 == a2:
        return None
    
    x_1 = int((b2 - b1) / (a1 - a2))
    y_1 = a1*x_1 + b1
    
    x_2 = x_1 + 1
    y_2 = a1*x_2 + b1
    
    x_3 = x_1 - 1
    y_3 = a1*x_3 + b1
    
    return (x_1, y_1), (x_2, y_2), (x_3, y_3)

In [12]:
def droites_par_losange(s: Sensor) -> tuple:
    
    xs = s.xy_s[0]
    ys = s.xy_s[1]
    r = manhattan_distance(s.xy_s, s.xy_b)
    
    a1 = a3 = -1
    a2 = a4 = 1
    
    b1 = ys + xs - r -1
    b2 = ys - xs - r -1
    b3 = ys + xs + r +1
    b4 = ys - xs + r +1
    
    return (a1, b1), (a2, b2), (a3, b3), (a4, b4)

In [13]:
def around_intersection(inter: tuple) -> tuple: # plus utilisée
    
    p1 = (inter[0]+1, inter[1])
    p2 = (inter[0], inter[1]+1)
    p3 = (inter[0]-1, inter[1])
    p4 = (inter[0], inter[1]-1)
    
    return p1, p2, p3, p4

In [14]:
def get_points_to_check(data: list, limit_min : int, limit_max : int) -> list:

    points_to_check = set()
    sensors = list_sensors(data)

    all_droites = []

    for sensor in sensors:
        d = droites_par_losange(sensor)
        for i in range(4):
            all_droites.append(d[i])

    
    for d1 in all_droites:
        for d2 in all_droites:
            intersection = intersection_droites(d1,d2)
            if intersection is not None:  
                for inter in intersection:
                    if inter[0] >= limit_min and inter[0] <= limit_max and inter[1] >= limit_min and inter[1] <= limit_max:
                        points_to_check.add(inter)

    return points_to_check

In [15]:
def get_distress_beacon(data: list, lim_min: int, lim_max: int) -> set:
    
    points_to_check = get_points_to_check(data, lim_min, lim_max)
    sensors = list_sensors(data)
    distress_beacon = []

    not_these_points = set()

    for point in points_to_check:
        for sensor in sensors:
            if manhattan_distance(point, sensor.xy_s) < manhattan_distance(sensor.xy_b, sensor.xy_s) + 1 :
                not_these_points.add(point)
                
    return list(points_to_check.difference(not_these_points))

In [16]:
def get_result_pt2(data: list, lim_min: int, lim_max: int) -> int:
    
    coord = get_distress_beacon(data, lim_min, lim_max)
    
    x = coord[0][0]
    y = coord[0][1]
    
    result = x * 4000000 + y
    
    return result

In [17]:
result_pt2 = get_result_pt2(data, 0, 4000000)

In [18]:
result_pt2 # résultat partie 2 : 12691026767556

12691026767556